In [1]:
import os
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_l = pd.read_csv('data/listings.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_l.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,gmaps_result,centre_transit_time
0,23163,https://www.airbnb.com/rooms/23163,20200229181007,2020-03-01,"Quiet 1BR directly in the Centre, Air-conditio...","Unique and elegant apartment rental in Prague,...",Unique and elegant apartment rental in Prague ...,"Unique and elegant apartment rental in Prague,...",none,Check my guidebook for this apartment to disco...,"To help you with your first steps in Prague, I...",The closest METRO station is called „Narodni t...,NaN,"Of course, you can contact me anytime during y...",Thank you for treating the property like it wa...,NaN,NaN,https://a0.muscache.com/im/pictures/01bbe32c-3...,NaN,5282,https://www.airbnb.com/users/show/5282,Klara,2008-12-17,"Prague, Prague, Czech Republic","Hello, \r\nglad to see that you are interested...",within an hour,97%,100%,t,https://a0.muscache.com/im/pictures/user/b7309...,https://a0.muscache.com/im/pictures/user/b7309...,Josefov,69.0,69.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Prague, Czech Republic, Czech Republic",Staré Město,Praha 1,NaN,Prague,Czech Republic,11000,Prague,"Prague, Czech Republic",CZ,Czech Republic,50.08229,14.41552,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",775.0,"$3,717.00",NaN,NaN,"$2,546.00",NaN,2,$636.00,1,365,1,7,365,365,5.6,365.0,today,t,19,40,70,288,2020-03-01,19,3,2010-09-20,2019-07-27,98.0,9.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,t,f,moderate,f,f,69,68,0,0,0.17,{'destination_addresses': ['Staroměstské nám. ...,9.883333
1,23169,https://www.airbnb.com/rooms/23169,20200229181007,2020-03-01,M302-Cozy Serviced Studio in Center,Masna studio offers a lot of space and privacy...,* CENTRAL LOCATION * EQUIPPED KITCHEN * BALCON...,Masna studio offers a lot of space and privacy...,none,Check my guidebook for this apartment to disco...,"To help you with your first steps in Prague, I...",The closest METRO station is called „Staromest...,NaN,"Of course, you can contact me anytime during y...",Thank you for treating the property like it wa...,NaN,NaN,https://a0.muscache.com/im/pictures/35353e66-2...,NaN,5282,https://www.airbnb.com/users/show/5282,Klara,2008-12-17,"Prague, Prague, Czech Republic","Hello, \r\nglad to see that you are interested...",within an hour,97

In [4]:
cols_to_drop = 'listing_url	scrape_id	last_scraped	name	summary	space	description	experiences_offered	neighborhood_overview	notes	transit	access	interaction	house_rules	thumbnail_url	medium_url	picture_url	xl_picture_url	host_url	host_name	host_location	host_about	host_thumbnail_url	host_picture_url	host_neighbourhood	host_listings_count	host_verifications	street	neighbourhood	city	state	zipcode	market	smart_location	country_code	country	latitude	longitude	is_location_exact	square_feet	weekly_price	monthly_price	maximum_nights	calendar_updated	has_availability	availability_30	availability_60	availability_90	availability_365	calendar_last_scraped	first_review	last_review	jurisdiction_names	require_guest_profile_picture	require_guest_phone_verification	calculated_host_listings_count'.split('\t')

In [5]:
df_lf = df_l.drop(cols_to_drop, axis = 1)

In [6]:
df_lf[['price', 'security_deposit', 'cleaning_fee', 'extra_people']].head()

,price,security_deposit,cleaning_fee,extra_people
0,"$3,717.00","$2,546.00",NaN,$636.00
1,"$2,493.00","$2,546.00",NaN,$636.00
2,"$2,540.00","$5,092.00",$458.00,$255.00
3,"$2,032.00","$3,819.00",$458.00,$204.00
4,"$1,039.00",$0.00,$350.00,$0.00


In [7]:
def convert_prices(df, cols):
    for col in cols:
        df[col] = df[col].str.replace('[$|,]', '').astype(float)
    return df

In [8]:
df_lf = convert_prices(df_lf, ['price', 'security_deposit', 'cleaning_fee', 'extra_people'])

In [9]:
df_a = pd.DataFrame(df_lf['amenities'].apply(lambda x: re.sub('[{|}|"]', '', x).split(',')))

In [10]:
df_a['id'] = df_lf['id']
df_a_con = pd.concat([pd.Series(row['id'], row['amenities']) for _, row in df_a.iterrows()]).reset_index()
df_a_con['val'] = 1
df_a_con.columns = ['amenity', 'id', 'value']

In [11]:
df_pivoted = df_a_con.pivot_table(index = 'id', columns = 'amenity', values = 'value', aggfunc='mean')

In [12]:
df_pivoted.head()

amenity,,toilet,24-hour check-in,Accessible-height bed,Accessible-height toilet,Air conditioning,Air purifier,Alfresco bathtub,Amazon Echo,BBQ grill,Baby bath,Baby monitor,Babysitter recommendations,Baking sheet,Balcony,Barbecue utensils,Bath towel,Bathroom essentials,Bathtub,Bathtub with bath chair,Beach essentials,Beach view,Beachfront,Bed linens,Bedroom comforts,Bidet,Body soap,Bread maker,Breakfast,Breakfast table,Building staff,Buzzer/wireless intercom,Cable TV,Carbon monoxide detector,Cat(s),Ceiling fan,Ceiling hoist,Central air conditioning,Changing table,Children’s books and toys,Children’s dinnerware,Cleaning before checkout,Coffee maker,Convection oven,Cooking basics,Crib,DVD player,Day bed,Disabled parking spot,Dishes and silverware,Dishwasher,Dog(s),Doorman,Double oven,Dryer,EV charger,Elevator,En suite bathroom,Espresso machine,Essentials,Ethernet connection,Extra pillows and blankets,Extra space around bed,Family/kid friendly,Fax machine,Fire extinguisher,Fire pit,Fireplace guards,Firm mattress,First aid kit,Fixed grab bars for shower,Fixed grab bars for toilet,Flat path to guest entrance,Formal dining area,Free parking on premises,Free street parking,Full kitchen,Game console,Garden or backyard,Gas oven,Gym,HBO GO,Hair dryer,Hammock,Handheld shower head,Hangers,Heat lamps,Heated floors,Heated towel rack,Heating,High chair,High-resolution computer monitor,Host greets you,Hot tub,Hot water,Hot water kettle,Indoor fireplace,Internet,Iron,Jetted tub,Keypad,Kitchen,Kitchenette,Lake access,Laptop friendly workspace,Lock on bedroom door,Lockbox,Long term stays allowed,Luggage dropoff allowed,Memory foam mattress,Microwave,Mini fridge,Mobile hoist,Mountain view,Mudroom,Murphy bed,Netflix,No stairs or steps to enter,Other,Other pet(s),Outdoor kitchen,Outdoor parking,Outdoor seating,Outlet covers,Oven,Pack ’n Play/travel crib,Paid parking off premises,Paid parking on premises,Patio or balcony,Pets allowed,Pets live on this property,Pillow-top mattress,Pocket wifi,Pool,Printer,Private entrance,Private hot tub,Private living room,Projector and screen,Rain shower,Refrigerator,Roll-in shower with chair,Room-darkening shades,Safety card,Sauna,Self check-in,Shampoo,Shared gym,Shared pool,Shower chair,Shower gel,Single level home,Ski-in/Ski-out,Smart TV,Smart lock,Smoke detector,Smoking allowed,Soaking tub,Sound system,Stair gates,Standing valet,Steam oven,Step-free shower,Stove,Suitable for events,Sun loungers,TV,Table corner guards,Terrace,Toilet paper,Trash can,Walk-in shower,Warming drawer,Washer,Waterfront,Well-lit path to entrance,Wheelchair accessible,Wide clearance to shower,Wide doorway to guest bathroom,Wide entrance,Wide entrance for guests,Wide entryway,Wide hallways,Wifi,Window guards,Wine cooler,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23163,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
23169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Na

In [13]:
df_pivoted_ratios = df_pivoted.isnull().sum() / df_pivoted.shape[0]

In [14]:
amenities_cols = df_pivoted_ratios[df_pivoted_ratios < 0.8].index.to_list()

In [15]:
amenities = df_pivoted[amenities_cols].fillna(0).reset_index()

In [16]:
amenities.head()

amenity,id,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi
0,23163,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23169,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,26748,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,26755,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,27916,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [17]:
df_all = df_lf.merge(amenities, on = 'id').drop('amenities', axis = 1)

In [18]:
features = df_all.isnull().sum() / df_all.shape[0]

In [19]:
df_all = df_all[features[features < 0.8].index.tolist()]

In [20]:
df_all.head()

,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,gmaps_result,centre_transit_time,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi
0,23163,5282,2008-12-17,within an hour,97%,100%,t,69.0,t,t,Praha 1,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,3717.0,2546.0,NaN,2,636.0,1,1,7,365,365,5.6,365.0,19,3,98.0,9.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,68,0,0,0.17,{'destination_addresses': ['Staroměstské nám. ...,9.883333,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23169,5282,2008-12-17,within an hour,97%,100%,t,69.0,t,t,Praha 1,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,2493.0,2546.0,NaN,2,636.0,1,1,7,365,365,1.2,365.0,113,1,95.0,10.0,10.0,10.0,10.0,10.0,9.0,f,t,f,moderate,68,0,0,0.95,{'destination_addresses': ['Staroměstské nám. ...,4.766667,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,26748,113902,2010-04-26,within an hour,100%,100%,t,3.0,t,t,Praha 7,Apartment,Entire home/apt,6,1.0,2.0,2.0,Real Bed,2540.0,5092.0,458.0,4,255.0,3,2,7,1125,1125,3.2,1125.0,345,54,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,3.00,{'destination_addresses': ['Staroměstské nám. ...,16.900000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,26755,113902,2010-04-26,within an hour,100%,100%,t,3.0,t,t,Praha 1,Apartment,Entire home/apt,4,1.5,1.0,2.0,Real Bed,2032.0,3819.0,458.0,3,204.0,3,2,7,1125,1125,3.2,1125.0,246,53,99.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,4.22,{'destination_addresses': ['Staroměstské nám. ...,13.450000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,27916,120384,2010-05-08,within an hour,100%,100%,t,3.0,t,t,Praha 3,Condominium,Entire home/apt,2,1.0,0.0,1.0,Real Bed,1039.0,0.0,350.0,1,0.0,2,2,2,50,50,2.0,50.0,411,48,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,3.52,{'destination_addresses': ['Staroměstské nám. ...,20.683333,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [21]:
df_all['host_response_rate'] = df_all['host_response_rate'].str.replace('%', '').astype(float)

In [22]:
df_all['host_acceptance_rate'] = df_all['host_acceptance_rate'].str.replace('%', '').astype(float)

In [23]:
df_all['host_since'] = df_all['host_since'].apply(pd.to_datetime)

In [24]:
for col in df_all.dtypes[df_all.dtypes == 'float'].index.tolist():
    df_all[col] = df_all[col].fillna(df_all[col].mean())

# Grouping neighbourhoods

In [25]:
df1 = df_all.groupby('neighbourhood_cleansed')['id'].count().reset_index()

In [26]:
df1['ratio'] = df1['id'] / df_all.shape[0]

In [27]:
df1['neighbourhood_grouped'] = df1.apply(lambda x: x['neighbourhood_cleansed'] if (x['ratio'] > 0.01) else 'Outskirts', axis = 1)

In [28]:
df1.groupby('neighbourhood_grouped')['ratio'].sum()

neighbourhood_grouped
Outskirts    0.051992
Praha 1      0.283929
Praha 10     0.052747
Praha 2      0.166690
Praha 3      0.133310
Praha 4      0.043269
Praha 5      0.087706
Praha 6      0.046703
Praha 7      0.056113
Praha 8      0.060852
Praha 9      0.016690
Name: ratio, dtype: float64

In [29]:
df_all = df_all.merge(df1[['neighbourhood_cleansed', 'neighbourhood_grouped']], on ='neighbourhood_cleansed')

In [30]:
df_all = df_all.drop(['neighbourhood_cleansed', 'gmaps_result', 'id', 'host_id'], axis = 1)

In [31]:
(pd.to_datetime('2020-03-01') - df_all['host_since'].iloc[0]).days

4092

In [32]:
df_all['host_since'] = df_all['host_since'].apply(lambda x: (pd.to_datetime('2020-03-01') - x).days)

In [33]:
df_all.head()

,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,centre_transit_time,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi,neighbourhood_grouped
0,4092.0,within an hour,97.0,100.0,t,69.0,t,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,3717.0,2546.0,508.480508,2,636.0,1,1,7,365,365,5.6,365.0,19,3,98.0,9.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,68,0,0,0.17,9.883333,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1
1,4092.0,within an hour,97.0,100.0,t,69.0,t,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,2493.0,2546.0,508.480508,2,636.0,1,1,7,365,365,1.2,365.0,113,1,95.0,10.0,10.0,10.0,10.0,10.0,9.0,f,t,f,moderate,68,0,0,0.95,4.766667,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1
2,3597.0,within an hour,100.0,100.0,t,3.0,t,t,Apartment,Entire home/apt,4,1.5,1.0,2.0,Real Bed,2032.0,3819.0,458.000000,3,204.0,3,2,7,1125,1125,3.2,1125.0,246,53,99.0,10.0,10.0,10.0,10.0,10.0,10.0,f,t,f,moderate,3,0,0,4.22,13.450000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1
3,4092.0,within an hour,97.0,100.0,t,69.0,t,t,Serviced apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,5472.0,5092.0,508.480508,4,636.0,1,1,7,365,365,1.8,365.0,12,3,93.0,9.0,10.0,9.0,9.0,10.0,9.0,f,t,f,moderate,68,0,0,0.11,5.250000,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1
4,3546.0,within a few hours,100.0,95.0,t,12.0,t,f,Apartment,Entire home/apt,3,1.0,1.0,1.0,Real Bed,1639.0,0.0,382.000000,2,255.0,3,3,3,365,365,3.0,365.0,276,32,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,f,strict_14_with_grace_period,12,0,0,2.40,8.850000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1


In [34]:
host_response_time = pd.get_dummies(df_all['host_response_time'], dummy_na = True, prefix = 'host_response_time')

In [35]:
df_all = df_all.merge(host_response_time, left_index = True, right_index = True).drop('host_response_time', axis = 1)

In [36]:
df_all = df_all.dropna()

In [37]:
for col in df_all.columns[df_all.dtypes == 'object'].tolist():
    print(col, df_all[col].unique())

host_is_superhost ['t' 'f']
host_has_profile_pic ['t' 'f']
host_identity_verified ['t' 'f']
property_type ['Apartment' 'Serviced apartment' 'Condominium' 'Bed and breakfast'
 'Guest suite' 'Loft' 'Aparthotel' 'House' 'Townhouse' 'Hotel'
 'Boutique hotel' 'Hostel' 'Other' 'Castle' 'Villa' 'Ryokan (Japan)'
 'Guesthouse' 'Houseboat' 'Campsite' 'Tiny house' 'Camper/RV' 'Cave'
 'Bungalow' 'Farm stay' 'Casa particular (Cuba)' 'In-law' 'Boat' 'Cottage'
 'Earth house' 'Cabin' 'Tent' 'Dome house']
room_type ['Entire home/apt' 'Private room' 'Hotel room' 'Shared room']
bed_type ['Real Bed' 'Pull-out Sofa' 'Futon' 'Airbed' 'Couch']
requires_license ['f']
instant_bookable ['t' 'f']
is_business_travel_ready ['f']
cancellation_policy ['moderate' 'strict_14_with_grace_period' 'super_strict_30' 'flexible'
 'strict' 'super_strict_60']
neighbourhood_grouped ['Praha 1' 'Praha 7' 'Praha 3' 'Praha 2' 'Outskirts' 'Praha 5' 'Praha 8'
 'Praha 4' 'Praha 10' 'Praha 6' 'Praha 9']


In [38]:
df_all = df_all.drop(['requires_license', 'is_business_travel_ready'], axis = 1)

In [39]:
df_all.head()

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,centre_transit_time,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi,neighbourhood_grouped,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan
0,4092.0,97.0,100.0,t,69.0,t,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,3717.0,2546.0,508.480508,2,636.0,1,1,7,365,365,5.6,365.0,19,3,98.0,9.0,10.0,10.0,10.0,10.0,10.0,t,moderate,68,0,0,0.17,9.883333,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0
1,4092.0,97.0,100.0,t,69.0,t,t,Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,2493.0,2546.0,508.480508,2,636.0,1,1,7,365,365,1.2,365.0,113,1,95.0,10.0,10.0,10.0,10.0,10.0,9.0,t,moderate,68,0,0,0.95,4.766667,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0
2,3597.0,100.0,100.0,t,3.0,t,t,Apartment,Entire home/apt,4,1.5,1.0,2.0,Real Bed,2032.0,3819.0,458.000000,3,204.0,3,2,7,1125,1125,3.2,1125.0,246,53,99.0,10.0,10.0,10.0,10.0,10.0,10.0,t,moderate,3,0,0,4.22,13.450000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0
3,4092.0,97.0,100.0,t,69.0,t,t,Serviced apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,5472.0,5092.0,508.480508,4,636.0,1,1,7,365,365,1.8,365.0,12,3,93.0,9.0,10.0,9.0,9.0,10.0,9.0,t,moderate,68,0,0,0.11,5.250000,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0
4,3546.0,100.0,95.0,t,12.0,t,f,Apartment,Entire home/apt,3,1.0,1.0,1.0,Real Bed,1639.0,0.0,382.000000,2,255.0,3,3,3,365,365,3.0,365.0,276,32,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,strict_14_with_grace_period,12,0,0,2.40,8.850000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,1,0,0


In [40]:
df2 = df_all.groupby('property_type')['host_since'].count().reset_index()
df2.columns = ['property_type', 'count']
df2['ratio'] = df2['count'] / df_all.shape[0]

In [41]:
df2.sort_values('ratio', ascending = False)

,property_type,count,ratio
1,Apartment,12121,0.832543
27,Serviced apartment,474,0.032557
24,Loft,372,0.025551
21,House,347,0.023834
19,Hostel,225,0.015454
12,Condominium,198,0.013600
17,Guest suite,190,0.013050
4,Boutique hotel,118,0.008105
0,Aparthotel,88,0.006044
2,Bed and breakfast,88,0.006044


In [42]:
df2['property_type_grouped'] = df2.apply(lambda x: x['property_type'] if x['ratio'] > 0.01 else 'Other', axis = 1)

In [43]:
df_all = df_all.merge(df2[['property_type', 'property_type_grouped']], on='property_type').drop('property_type', axis = 1)

In [44]:
df_all.groupby('property_type_grouped').count()

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,centre_transit_time,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi,neighbourhood_grouped,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan
property_type_grouped,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Apartment,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121,12121
Condominium,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198,198
Guest suite,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190,190
Hostel,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225,225
House,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347,347
Loft,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372,372
Other,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,632,

In [45]:
df_all.head()

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,centre_transit_time,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi,neighbourhood_grouped,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,property_type_grouped
0,4092.0,97.0,100.0,t,69.0,t,t,Entire home/apt,4,1.0,1.0,2.0,Real Bed,3717.0,2546.0,508.480508,2,636.0,1,1,7,365,365,5.6,365.0,19,3,98.0,9.0,10.0,10.0,10.0,10.0,10.0,t,moderate,68,0,0,0.17,9.883333,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0,Apartment
1,4092.0,97.0,100.0,t,69.0,t,t,Entire home/apt,4,1.0,1.0,2.0,Real Bed,2493.0,2546.0,508.480508,2,636.0,1,1,7,365,365,1.2,365.0,113,1,95.0,10.0,10.0,10.0,10.0,10.0,9.0,t,moderate,68,0,0,0.95,4.766667,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0,Apartment
2,3597.0,100.0,100.0,t,3.0,t,t,Entire home/apt,4,1.5,1.0,2.0,Real Bed,2032.0,3819.0,458.000000,3,204.0,3,2,7,1125,1125,3.2,1125.0,246,53,99.0,10.0,10.0,10.0,10.0,10.0,10.0,t,moderate,3,0,0,4.22,13.450000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0,Apartment
3,3546.0,100.0,95.0,t,12.0,t,f,Entire home/apt,3,1.0,1.0,1.0,Real Bed,1639.0,0.0,382.000000,2,255.0,3,3,3,365,365,3.0,365.0,276,32,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,strict_14_with_grace_period,12,0,0,2.40,8.850000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,1,0,0,Apartment
4,3502.0,67.0,65.0,f,1.0,t,f,Entire home/apt,3,1.0,1.0,2.0,Real Bed,2170.0,3182.0,1018.000000,2,764.0,3,3,3,365,365,3.0,365.0,321,6,95.0,10.0,10.0,10.0,10.0,10.0,10.0,f,strict_14_with_grace_period,1,0,0,2.76,10.266667,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,1,0,0,0,Apartment


In [46]:
property_type = pd.get_dummies(df_all['property_type_grouped'], prefix = 'property_type')

In [47]:
df_all = df_all.merge(property_type, left_index = True, right_index = True).drop('property_type_grouped', axis = 1)

In [48]:
df_all['bed_type'] = df_all['bed_type'].apply(lambda x: 1 if (x == 'Real Bed') else 0)

In [49]:
df_all.dtypes[df_all.dtypes == 'object']

host_is_superhost         object
host_has_profile_pic      object
host_identity_verified    object
room_type                 object
instant_bookable          object
cancellation_policy       object
neighbourhood_grouped     object
dtype: object

In [50]:
room_type = pd.get_dummies(df_all['room_type'], prefix = 'room_type')
df_all = df_all.merge(room_type, left_index = True, right_index = True).drop('room_type', axis = 1)

In [51]:
df_all.head()

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,centre_transit_time,Bed linens,Carbon monoxide detector,Coffee maker,Cooking basics,Dishes and silverware,Dishwasher,Dryer,Elevator,Essentials,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,First aid kit,Hair dryer,Hangers,Heating,Host greets you,Hot water,Internet,Iron,Kitchen,Laptop friendly workspace,Lock on bedroom door,Long term stays allowed,Luggage dropoff allowed,Microwave,Oven,Paid parking off premises,Pets allowed,Private entrance,Refrigerator,Shampoo,Smoke detector,Stove,TV,Washer,Wifi,neighbourhood_grouped,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,property_type_Apartment,property_type_Condominium,property_type_Guest suite,property_type_Hostel,property_type_House,property_type_Loft,property_type_Other,property_type_Serviced apartment,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,4092.0,97.0,100.0,t,69.0,t,t,4,1.0,1.0,2.0,1,3717.0,2546.0,508.480508,2,636.0,1,1,7,365,365,5.6,365.0,19,3,98.0,9.0,10.0,10.0,10.0,10.0,10.0,t,moderate,68,0,0,0.17,9.883333,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0
1,4092.0,97.0,100.0,t,69.0,t,t,4,1.0,1.0,2.0,1,2493.0,2546.0,508.480508,2,636.0,1,1,7,365,365,1.2,365.0,113,1,95.0,10.0,10.0,10.0,10.0,10.0,9.0,t,moderate,68,0,0,0.95,4.766667,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0
2,3597.0,100.0,100.0,t,3.0,t,t,4,1.5,1.0,2.0,1,2032.0,3819.0,458.000000,3,204.0,3,2,7,1125,1125,3.2,1125.0,246,53,99.0,10.0,10.0,10.0,10.0,10.0,10.0,t,moderate,3,0,0,4.22,13.450000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0
3,3546.0,100.0,95.0,t,12.0,t,f,3,1.0,1.0,1.0,1,1639.0,0.0,382.000000,2,255.0,3,3,3,365,365,3.0,365.0,276,32,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,strict_14_with_grace_period,12,0,0,2.40,8.850000,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,3502.0,67.0,65.0,f,1.0,t,f,3,1.0,1.0,2.0,1,2170.0,3182.0,1018.000000,2,764.0,3,3,3,365,365,3.0,365.0,321,6,95.0,10.0,10.0,10.0,10.0,10.0,10.0,f,strict_14_with_grace_period,1,0,0,2.76,10.266667,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Praha 1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [52]:
neighbourhood = pd.get_dummies(df_all['neighbourhood_grouped'], prefix = 'neighbourhood')
df_all = df_all.merge(neighbourhood, left_index = True, right_index = True).drop('neighbourhood_grouped', axis = 1)

In [56]:
df_all['cancellation_policy'] = df_all['cancellation_policy'].apply(lambda x: 'strict' if ('strict' in x) else x)

In [59]:
cancellation_policy = pd.get_dummies(df_all['cancellation_policy'], prefix = 'cancellation_policy')
df_all = df_all.merge(cancellation_policy, left_index = True, right_index = True).drop('cancellation_policy', axis = 1)

In [71]:
df_all = df_all.drop('host_has_profile_pic', axis = 1)

In [72]:
for col in df_all.dtypes[df_all.dtypes == 'object'].index.tolist():
    df_all[col] = df_all[col].replace({'t': 1, 'f': 0})

f    9993
t    4566
Name: host_is_superhost, dtype: int64
f    9823
t    4736
Name: host_identity_verified, dtype: int64
t    10217
f     4342
Name: instant_bookable, dtype: int64


In [75]:
df_all['host_is_superhost'].replace({'t': 1, 'f': 0})

dtype('int64')

In [76]:
for col in df_all.dtypes[df_all.dtypes == 'object'].index.tolist():
    df_all[col] = df_all[col].replace({'t': 1, 'f': 0})

In [78]:
df_all.shape

(14559, 106)

# Fitting a model

In [114]:
import sklearn.model_selection
from xgboost.sklearn import XGBRegressor
import matplotlib.pyplot as plt

In [80]:
y = df_all['price']

In [81]:
X = df_all.drop('price', axis = 1)

In [88]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=42)

In [101]:
eval_set = [(X_train, y_train), (X_test, y_test)]

In [116]:
gbm = XGBRegressor(n_estimators=1000)

In [120]:
m = gbm.fit(X_train, y_train, early_stopping_rounds=50, eval_set = eval_set)

[17:54:50] DEBUG: /workspace/src/gbm/gbtree.cc:146: Using tree method: 2
[17:54:50] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=6
[0]	validation_0-rmse:11791.97461	validation_1-rmse:11164.99316
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[17:54:50] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=6
[1]	validation_0-rmse:10524.52832	validation_1-rmse:10177.24414
[17:54:50] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[2]	validation_0-rmse:9756.54981	validation_1-rmse:9754.45410
[17:54:50] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=6
[3]	validation_0-rmse:9067.28027	validation_1-rmse:9349.27441
[17:54:50] INFO: /workspace/src/tree

[43]	validation_0-rmse:3423.51465	validation_1-rmse:7372.58691
[17:54:52] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=6
[44]	validation_0-rmse:3306.73901	validation_1-rmse:7379.68066
[17:54:52] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=6
[45]	validation_0-rmse:3196.09570	validation_1-rmse:7345.00195
[17:54:52] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=6
[46]	validation_0-rmse:3177.35791	validation_1-rmse:7344.68262
[17:54:52] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=6
[47]	validation_0-rmse:3089.09253	validation_1-rmse:7342.19238
[17:54:52] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 94 extra nodes, 0 pruned nodes, max_depth=6
[48]	validation_0-rmse:3075.71851	validation_1-rmse:7337.37695
[17:54:52] INFO: /wo

[17:54:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=6
[89]	validation_0-rmse:2032.59326	validation_1-rmse:7227.37109
[17:54:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 80 extra nodes, 0 pruned nodes, max_depth=6
[90]	validation_0-rmse:2027.18384	validation_1-rmse:7232.32617
[17:54:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 120 extra nodes, 0 pruned nodes, max_depth=6
[91]	validation_0-rmse:1988.37390	validation_1-rmse:7230.34473
[17:54:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 96 extra nodes, 0 pruned nodes, max_depth=6
[92]	validation_0-rmse:1966.38965	validation_1-rmse:7231.68506
[17:54:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=6
[93]	validation_0-rmse:1961.33057	validation_1-rmse:7231.23047
[17:54:54] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 80 extra

[17:54:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 72 extra nodes, 0 pruned nodes, max_depth=6
[133]	validation_0-rmse:1411.96143	validation_1-rmse:7177.79004
[17:54:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[134]	validation_0-rmse:1408.54150	validation_1-rmse:7177.04541
[17:54:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[135]	validation_0-rmse:1397.03979	validation_1-rmse:7175.89404
[17:54:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[136]	validation_0-rmse:1391.07361	validation_1-rmse:7169.44971
[17:54:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=6
[137]	validation_0-rmse:1387.61462	validation_1-rmse:7169.80273
[17:54:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 86

[17:54:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=6
[177]	validation_0-rmse:1123.48267	validation_1-rmse:7148.69678
[17:54:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=6
[178]	validation_0-rmse:1114.58447	validation_1-rmse:7149.88965
[17:54:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=6
[179]	validation_0-rmse:1103.29981	validation_1-rmse:7147.96728
[17:54:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 112 extra nodes, 0 pruned nodes, max_depth=6
[180]	validation_0-rmse:1094.44177	validation_1-rmse:7146.53223
[17:54:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[181]	validation_0-rmse:1081.96423	validation_1-rmse:7141.93164
[17:54:58] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 

[17:55:00] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 118 extra nodes, 0 pruned nodes, max_depth=6
[222]	validation_0-rmse:887.98261	validation_1-rmse:7134.00342
[17:55:00] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=6
[223]	validation_0-rmse:882.23950	validation_1-rmse:7133.75977
[17:55:01] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=6
[224]	validation_0-rmse:876.94678	validation_1-rmse:7134.03857
[17:55:01] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 96 extra nodes, 0 pruned nodes, max_depth=6
[225]	validation_0-rmse:872.88318	validation_1-rmse:7133.10547
[17:55:01] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=6
[226]	validation_0-rmse:869.68573	validation_1-rmse:7131.91504
[17:55:01] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 ex

[17:55:03] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=6
[267]	validation_0-rmse:739.10657	validation_1-rmse:7126.87012
[17:55:03] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=6
[268]	validation_0-rmse:738.61841	validation_1-rmse:7126.89990
[17:55:03] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 106 extra nodes, 0 pruned nodes, max_depth=6
[269]	validation_0-rmse:736.47211	validation_1-rmse:7126.87402
[17:55:03] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 116 extra nodes, 0 pruned nodes, max_depth=6
[270]	validation_0-rmse:729.05109	validation_1-rmse:7126.97510
[17:55:03] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=6
[271]	validation_0-rmse:724.96558	validation_1-rmse:7126.53369
[17:55:03] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 112 ex

[311]	validation_0-rmse:595.50696	validation_1-rmse:7128.47803
[17:55:05] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[312]	validation_0-rmse:594.37390	validation_1-rmse:7128.57520
[17:55:05] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=6
[313]	validation_0-rmse:593.61932	validation_1-rmse:7128.42334
[17:55:05] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[314]	validation_0-rmse:591.39160	validation_1-rmse:7128.25147
[17:55:05] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[315]	validation_0-rmse:588.85480	validation_1-rmse:7127.87598
[17:55:05] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 102 extra nodes, 0 pruned nodes, max_depth=6
[316]	validation_0-rmse:587.74579	validation_1-rmse:7127.85791
[17:55:05] INFO: 

In [125]:
def plot_n_estimators(model):
    results = model.evals_result()
    epochs = len(results['validation_0']['rmse'])
    x_axis = range(0, epochs)

    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
    ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
    ax.legend()
    plt.ylabel('RMSE')
    plt.title('XGBoost RMSE')
    plt.show()

In [126]:
plot_n_estimators(m)

TypeError: plot_n_estimators() missing 1 required positional argument: 'model'